In [ ]:
!pip install 

In [2]:
import importlib

In [1]:
import pandas as pd
import numpy as np
import re
import os
import pdb
import dask
from datetime import datetime
import dask.dataframe as dd
import dask
from dask import delayed,compute
import sys 
current_dir = os.getcwd()
loading_dir = os.path.join(current_dir,"data","raw",
                           "flash_crash_DJIA","csv_files")
saving_dir = os.path.join(current_dir,"data","clean","flash_crash")
src_path = current_dir + "/src"
sys.path.append(src_path)

/Users/ilyesbenayed/anaconda3/envs/bigData/lib/python3.11/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


In [3]:
importlib.reload(loading_parallel)

NameError: name 'loading_parallel' is not defined

In [4]:
import loading_parallel


#### Testing at first the extraction function : 

In [5]:
ticker = "AAPL.OQ-2010"
bbo_files, trade_files = loading_parallel.extract_files(loading_dir,
                                       ticker,
                                       "2010-01-01",
                                       "2010-01-05")

In [6]:
print(trade_files)
print(bbo_files)

['/Users/ilyesbenayed/Desktop/Big data/data/raw/flash_crash_DJIA/csv_files/AAPL.OQ-2010/trade/2010_01/2010-01-01-AAPL.OQ-trade.csv.gz', '/Users/ilyesbenayed/Desktop/Big data/data/raw/flash_crash_DJIA/csv_files/AAPL.OQ-2010/trade/2010_01/2010-01-04-AAPL.OQ-trade.csv.gz', '/Users/ilyesbenayed/Desktop/Big data/data/raw/flash_crash_DJIA/csv_files/AAPL.OQ-2010/trade/2010_01/2010-01-05-AAPL.OQ-trade.csv.gz']
['/Users/ilyesbenayed/Desktop/Big data/data/raw/flash_crash_DJIA/csv_files/AAPL.OQ-2010/bbo/2010_01/2010-01-05-AAPL.OQ-bbo.csv.gz', '/Users/ilyesbenayed/Desktop/Big data/data/raw/flash_crash_DJIA/csv_files/AAPL.OQ-2010/bbo/2010_01/2010-01-04-AAPL.OQ-bbo.csv.gz', '/Users/ilyesbenayed/Desktop/Big data/data/raw/flash_crash_DJIA/csv_files/AAPL.OQ-2010/bbo/2010_01/2010-01-01-AAPL.OQ-bbo.csv.gz']


## The function works well : now time to test loadTRTH

#### This is an example on how to read one TRTH file : 

In [7]:
trth_file = loading_parallel.load_TRTH_trade(trade_files[1])
compute(trth_file)

(                                     trade_price  trade_volume
 xltime                                                        
 2010-01-04 09:30:00.576999424-05:00     213.5250           400
 2010-01-04 09:30:00.616000-05:00        213.5375          1130
 2010-01-04 09:30:00.635999488-05:00     213.5000           100
 2010-01-04 09:30:00.648999936-05:00     213.5500           100
 2010-01-04 09:30:00.676999680-05:00     213.5000           100
 ...                                          ...           ...
 2010-01-04 15:59:59.526999808-05:00     214.2900          1231
 2010-01-04 15:59:59.537999872-05:00     214.2800          2169
 2010-01-04 15:59:59.746000128-05:00     214.3300           700
 2010-01-04 15:59:59.828999680-05:00     214.2900           800
 2010-01-04 15:59:59.988999936-05:00     214.2700          1901
 
 [18379 rows x 2 columns],)

#### Example of reading too many files and concatenate them together : 

In [8]:
def read_TRTH_files(trade_files):
    delayed_dfs = []
    for file in trade_files:
        delayed_df = delayed(loading_parallel.load_TRTH_trade)(file,
                                              only_regular_trading_hours=True,
                                              merge_sub_trades=True,
                                              drop_columns=True)
        delayed_dfs.append(delayed_df)
    dfs = compute(*delayed_dfs)
    final_df = pd.concat(dfs, ignore_index=False)
    final_df = final_df.sort_index()
    return final_df

In [9]:
%%time 
read_TRTH_files(trade_files)

CPU times: user 9.66 ms, sys: 13 ms, total: 22.7 ms
Wall time: 459 ms


,trade_price,trade_volume
xltime,,
2010-01-04 09:30:00.576999424-05:00,213.5250,400
2010-01-04 09:30:00.616000-05:00,213.5375,1130
2010-01-04 09:30:00.635999488-05:00,213.5000,100
2010-01-04 09:30:00.648999936-05:00,213.5500,100
2010-01-04 09:30:00.676999680-05:00,213.5000,100
...,...,...
2010-01-05 15:59:59.070999296-05:00,214.3200,396
2010-01-05 15:59:59.480999424-05:00,214.3700,685
2010-01-05 15:59:59.481999616-05:00,214.3700,815


#### Remark: The function works well with multiple entries : 
Let's measure the time to read January month 

In [10]:
bbo_files, trade_files = loading_parallel.extract_files(loading_dir,
                                       ticker,
                                       "2010-01-01",
                                       "2010-01-31")

In [11]:
%%time 
read_TRTH_files(trade_files)

CPU times: user 44.2 ms, sys: 29.8 ms, total: 74 ms
Wall time: 1.02 s


,trade_price,trade_volume
xltime,,
2010-01-04 09:30:00.576999424-05:00,213.525000,400.0
2010-01-04 09:30:00.616000-05:00,213.537500,1130.0
2010-01-04 09:30:00.635999488-05:00,213.500000,100.0
2010-01-04 09:30:00.648999936-05:00,213.550000,100.0
2010-01-04 09:30:00.676999680-05:00,213.500000,100.0
...,...,...
2010-01-29 15:59:59.755999232-05:00,192.155000,300.0
2010-01-29 15:59:59.772000-05:00,192.000000,200.0
2010-01-29 15:59:59.813999616-05:00,192.185714,4048.0


#### Remark: Everithing works well, the program seems to be efficent : 
We read the data fast : (1 second to read more than 1  million data )

## BBO testing function 

In [12]:
bbo_example = loading_parallel.load_TRTH_bbo(bbo_files[0])
compute(bbo_example)

(                                     bid-price  bid-volume  ask-price  \
 xltime                                                                  
 2010-01-20 09:30:00.012998912-05:00     214.75           1     214.91   
 2010-01-20 09:30:00.014999296-05:00     214.80           1     214.93   
 2010-01-20 09:30:00.130999552-05:00     214.80           1     214.90   
 2010-01-20 09:30:00.142999552-05:00     214.80           1     214.93   
 2010-01-20 09:30:00.150999808-05:00     214.80           2     214.95   
 ...                                        ...         ...        ...   
 2010-01-20 15:59:59.785999872-05:00     211.64          26     211.65   
 2010-01-20 15:59:59.790999552-05:00     211.64          26     211.65   
 2010-01-20 15:59:59.889999360-05:00     211.64          26     211.65   
 2010-01-20 15:59:59.952999424-05:00     211.64          26     211.65   
 2010-01-20 15:59:59.996999424-05:00     211.64          26     211.65   
 
                                    

#### Now time to test on multiple bbo files 

In [13]:
def read_bbo_files(trade_files):
    delayed_dfs = []
    for file in trade_files:
        delayed_df = delayed(loading_parallel.load_TRTH_bbo)(file)
        delayed_dfs.append(delayed_df)
    dfs = compute(*delayed_dfs)
    final_df = pd.concat(dfs, ignore_index=False)
    final_df = final_df.sort_index()
    return final_df

In [14]:
%%time 
read_bbo_files(bbo_files)

CPU times: user 122 ms, sys: 114 ms, total: 236 ms
Wall time: 1.53 s


,bid-price,bid-volume,ask-price,ask-volume
xltime,,,,
2010-01-04 09:30:00.000999936-05:00,213.32,1,213.42,1
2010-01-04 09:30:00.009000192-05:00,213.37,1,213.50,3
2010-01-04 09:30:00.020000-05:00,213.38,2,213.50,1
2010-01-04 09:30:00.042000128-05:00,213.39,1,213.50,1
2010-01-04 09:30:00.087999744-05:00,213.40,1,213.50,1
...,...,...,...,...
2010-01-29 15:59:59.845999616-05:00,191.97,2,191.98,51
2010-01-29 15:59:59.889999360-05:00,191.97,2,191.98,50
2010-01-29 15:59:59.932999936-05:00,191.97,1,191.98,50


#### Everything seems good : We are well using the load_bbo files 

## Now the big function to save: 

In [18]:
@dask.delayed
def load_merge_trade_bbo(ticker,date_start,
                         date_end,
                         dirBase,
                         suffix="parquet",
                         suffix_save=None,
                         dirSaveBase = None ,
                         saveOnly=False,
                         doSave=False):
    """
    This function allows to merge trade and bbo together of a given ticker, 
    We should precise the destination to save the merged data 
    The date merged can be done in any period we would like to test on
    By default the method returns the merged series, but can save or only save the merged series 
    Gien a location of the saving directory, the suffix is by default a parquet file. 
    """
    bbo_files,trade_files  = loading_parallel.extract_files(dirBase,ticker,date_start,date_end)
    trades = read_TRTH_files(trade_files)
    bbos = read_bbo_files(bbo_files)
    try:
        trades.shape + bbos.shape
    except:
        return None
    
    events=trades.join(bbos,how="outer")
    
    if doSave:
        dirSave=os.path.join(dirSaveBase,ticker)
        if not os.path.isdir(dirSave):
            os.makedirs(dirSave)

        if suffix_save:
            suffix=suffix_save
        
        file_events=os.path.join(dirSave , date_start + date_end)
       # pdb.set_trace()

        saved=False
        if suffix == "arrow":
            # Convert Pandas DataFrame to Dask DataFrame
            dask_df = dd.from_pandas(events, npartitions=1)  # Adjust npartitions based on your dataset size and memory
            # Dask doesn't directly support exporting to Arrow, so you would need to use PyArrow separately if needed
            # For now, you can save it as Parquet which is compatible with Arrow
            dask_df.to_parquet(file_events+"."+suffix, write_index=True)  # Parquet format is compatible with Arrow
            saved = True
        
        elif suffix == "parquet":
            dask_df = dd.from_pandas(events, npartitions=1)  # Convert Pandas DataFrame to Dask DataFrame
            dask_df.to_parquet(file_events, write_index=True, engine='pyarrow', write_metadata_file=True)
            saved = True
        
        if not saved:
            print(f"suffix {suffix}: format not recognized")

        if saveOnly:
            return saved
    return events




In [23]:

result = load_merge_trade_bbo(
    ticker="AAPL.OQ-2010",
    date_start="2010-01-01",
    date_end="2010-01-31",
    dirBase=loading_dir,
    suffix="parquet",
    suffix_save=None,
    dirSaveBase=saving_dir,
    saveOnly=False,
    doSave=True
)


In [25]:
!pip install pyarrow==12.0.1
!pip install cchardet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.6/22.6 MB 849.0 kB/s eta 0:00:0000:0100:01
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 11.0.0
    Uninstalling pyarrow-11.0.0:
      Successfully uninstalled pyarrow-11.0.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 653.6/653.6 kB 2.3 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [23 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.macosx-11.1-arm64-cpython-311
      creating build/lib.macosx-11.1-arm64-cpython-311/cchardet
      copying src/cchardet/version.py -> build/lib.macosx-11.1-arm64-cpython-311/cchardet
      copying src/cchardet/__init__.py -> build/lib.macosx-11.1-arm64-cpython-311/cchardet
      running build_ext
      building 'cchardet._cchardet' extension
      creating bu

In [24]:
%%time
compute(result)

/Users/ilyesbenayed/anaconda3/envs/bigData/lib/python3.11/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(
/Users/ilyesbenayed/anaconda3/envs/bigData/lib/python3.11/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


RuntimeError: file metadata is only available after writer close

#### Now reading the file arrow: 

In [125]:
checking_dir = os.path.join(saving_dir,"AAPL.OQ-2010","2010-01-012010-01-31.arrow")

In [126]:
os.listdir(checking_dir)

['part.0.parquet']

In [149]:

df = pd.read_parquet(os.path.join(checking_dir,
                                  "part.0.parquet"),
                     engine='pyarrow', 
                     convert_dates=False)



ValueError: pyarrow.lib.IpcWriteOptions size changed, may indicate binary incompatibility. Expected 88 from C header, got 72 from PyObject

In [141]:
import pyarrow.parquet as pq

parquet_file = pq.ParquetFile(os.path.join(checking_dir, "part.0.parquet"))
print(parquet_file.schema)


ValueError: pyarrow.lib.IpcWriteOptions size changed, may indicate binary incompatibility. Expected 88 from C header, got 72 from PyObject

In [140]:
!pip install --upgrade pandas pyarrow fastparquet



  Using cached pyarrow-14.0.2-cp311-cp311-macosx_11_0_arm64.whl.metadata (3.0 kB)
Using cached pyarrow-14.0.2-cp311-cp311-macosx_11_0_arm64.whl (24.0 MB)
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 11.0.0
    Uninstalling pyarrow-11.0.0:
      Successfully uninstalled pyarrow-11.0.0


In [131]:
!pip install fastparquet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 682.2/682.2 kB 3.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 8.2 MB/s eta 0:00:0000:0100:01m
